In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_SERIALIZABLE
from tabulate import tabulate

con = psycopg2.connect(
    host= "localhost",
    database= "postgres",
    user= "postgres",
    password= "S@iteja2"
  )

print(con)

<connection object at 0x000001B2668F87B0; dsn: 'user=postgres password=xxx dbname=postgres host=localhost', closed: 0>


In [2]:
def execute_transaction(commands):
    con = None
    try:
        con = psycopg2.connect(
        host= "localhost",
        database= "postgres",
        user= "postgres",
        password= "S@iteja2"
    )
        con.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_SERIALIZABLE)  # Serializable isolation level
        cur = con.cursor()

        # Executing each command in the transaction
        for command in commands:
            cur.execute(command)

        # Commit the transaction if all commands execute successfully
        con.commit()
        print("Transaction successfully committed.")

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error occurred:", error)
        # Rollback in case of error
        if con:
            con.rollback()
            print("Transaction rolled back.")

    finally:
        if cur:
            cur.close()
        if con:
            con.close()
        print("Database connection is now closed.")

In [3]:
drop_tables = """
DROP TABLE IF EXISTS stock;
DROP TABLE IF EXISTS product;
DROP TABLE IF EXISTS depot;
"""
execute_transaction([drop_tables])

Transaction successfully committed.
Database connection is now closed.


In [4]:
create_tables = """

CREATE TABLE IF NOT EXISTS product (
    prodid VARCHAR(4),
    pname VARCHAR(50),
    price DECIMAL(10, 2)
);

CREATE TABLE IF NOT EXISTS depot (
    depid VARCHAR(5),
    addr VARCHAR(50),
    volume INT
);

CREATE TABLE IF NOT EXISTS stock (
    prodid VARCHAR(4),
    depid VARCHAR(4),
    quantity INT
);
"""

execute_transaction([create_tables])

Transaction successfully committed.
Database connection is now closed.


In [5]:
alter_commands = """
ALTER TABLE Product ADD CONSTRAINT pk_product PRIMARY KEY (prodid);
ALTER TABLE Product ADD CONSTRAINT ck_product_price CHECK (price > 0);
ALTER TABLE depot ADD CONSTRAINT pk_depot PRIMARY KEY (depid);
ALTER TABLE stock ADD CONSTRAINT pk_stock PRIMARY KEY (prodid,depid);
ALTER TABLE stock ADD CONSTRAINT fk_product FOREIGN KEY (prodid) REFERENCES product(prodid);
ALTER TABLE stock ADD CONSTRAINT fk_depot FOREIGN KEY (depid) REFERENCES depot(depid);
"""
execute_transaction([alter_commands])

Transaction successfully committed.
Database connection is now closed.


In [6]:
insert_values = """
INSERT INTO product (prodid, pname, price)
VALUES 
    ('p1', 'tape', 2.5),
    ('p2', 'tv', 250),
    ('p3', 'vcr', 80);

INSERT INTO depot (depid, addr, volume)
VALUES ('d1', 'NEW YORK', 9000),
       ('d2', 'Syracuse', 6000),
       ('d4', 'NEW YORK', 2000);

INSERT INTO stock (prodid, depid, quantity)
VALUES 
    ('p1', 'd1', 1000),
    ('p1', 'd2', -100),
    ('p1', 'd4', 1200),
    ('p3', 'd1', 3000),
    ('p3', 'd4', 2000),
    ('p2', 'd4', 1500),
    ('p2', 'd1', -400),
    ('p2', 'd2', 2000);
"""
execute_transaction([insert_values])

Transaction successfully committed.
Database connection is now closed.


In [7]:
transaction_2 = [
    "ALTER TABLE Stock DROP CONSTRAINT IF EXISTS fk_product;"
    "UPDATE stock SET prodid = 'pp1' WHERE prodid = 'p1';"
    "UPDATE product SET prodid = 'pp1' WHERE prodid = 'p1';"
    "ALTER TABLE Stock ADD CONSTRAINT fk_product FOREIGN KEY (prodid) REFERENCES product(prodid) DEFERRABLE INITIALLY IMMEDIATE;"
]

"Rollback;"

execute_transaction(transaction_2)

Transaction successfully committed.
Database connection is now closed.


In [8]:
transaction_1= [
    
    "ALTER TABLE Stock DROP CONSTRAINT IF EXISTS fk_depot;"
    "UPDATE stock SET depid = 'dd1' WHERE depid = 'd1';"
    "UPDATE depot SET depid = 'dd1' WHERE depid = 'd1';"
    "ALTER TABLE Stock ADD CONSTRAINT fk_depot FOREIGN KEY (depid) REFERENCES depot(depid) DEFERRABLE INITIALLY IMMEDIATE;"
]

"Rollback;"

execute_transaction(transaction_1)

Transaction successfully committed.
Database connection is now closed.
